In [1]:
import pandas as pd
import mygene

In [2]:
df = pd.read_csv('/data/sandbox/test_fpkm_format.tsv.gz', sep='\t')
df

,gene_id,read_count,fpkm,fpkm_uq
0,ENSG00000223972.5,0,0.000000,0.000000e+00
1,ENSG00000227232.5,120,1.429815,3.190485e+04
2,ENSG00000278267.1,70,16.570780,3.697600e+05
3,ENSG00000243485.3,0,0.000000,0.000000e+00
4,ENSG00000274890.1,0,0.000000,0.000000e+00
...,...,...,...,...
60478,ENSG00000198695.2,11796,361.683983,8.070608e+06
60479,ENSG00000210194.1,62,14.464266,3.227553e+05
60480,ENSG00000198727.2,129835,1831.723653,4.087304e+07
60481,ENSG00000210195.2,194,47.316391,1.055817e+06


In [3]:
def convert_to_hugo(expression_df):
    expression_df.index = [x.split('.')[0] for x in expression_df.index]
    expression_df.index.name = 'gene_id'
    expression_df = expression_df.groupby('gene_id').mean()
    
    mg = mygene.MyGeneInfo()
    genes = list(expression_df.index)
    
    results = mg.querymany(genes, scopes='ensemblgene', fields='symbol', species='human')
    gene_id_to_symbol = {d['query']: d['symbol'] for d in results if 'symbol' in d}
    
    mask = np.asarray([True if gene_id in gene_id_to_symbol else False for gene_id in expression_df.index])
    expression_df = expression_df[mask]
    expression_df['gene_symbol'] = [gene_id_to_symbol[gene_id] for gene_id in expression_df.index]
    
    expression_df = expression_df.set_index('gene_symbol')
    expression_df.index.name = 'gene'
    expression_df = expression_df.groupby('gene').mean()
    
    return expression_df

In [4]:
mg = mygene.MyGeneInfo()


In [6]:
genes = sorted(set([g.split('.')[0] for g in df['gene_id'].to_list()]))
results = mg.querymany(genes, scopes='ensemblgene', fields='symbol', species='human')

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [7]:
gene_id_to_symbol = {d['query']: d['symbol'] for d in results if 'symbol' in d}
len(gene_id_to_symbol)

40600

In [14]:
df['gene_symbol'] = [gene_id_to_symbol.get(g.split('.')[0], '')
                     for g in df['gene_id'].to_list()]
df = df[['gene_id', 'gene_symbol', 'read_count', 'fpkm', 'fpkm_uq']]

In [15]:
df

,gene_id,gene_symbol,read_count,fpkm,fpkm_uq
0,ENSG00000223972.5,DDX11L1,0,0.000000,0.000000e+00
1,ENSG00000227232.5,WASH7P,120,1.429815,3.190485e+04
2,ENSG00000278267.1,MIR6859-1,70,16.570780,3.697600e+05
3,ENSG00000243485.3,MIR1302-2HG,0,0.000000,0.000000e+00
4,ENSG00000274890.1,,0,0.000000,0.000000e+00
...,...,...,...,...,...
60478,ENSG00000198695.2,ND6,11796,361.683983,8.070608e+06
60479,ENSG00000210194.1,MT-TE,62,14.464266,3.227553e+05
60480,ENSG00000198727.2,CYTB,129835,1831.723653,4.087304e+07
60481,ENSG00000210195.2,MT-TT,194,47.316391,1.055817e+06
